In [1]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
from logger import Logger 
from datetime import datetime
import tqdm 
import pandas as pd
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [2]:
# Prepare source dataset 

source_name = 'data/penalty/Penalty_size_{size}_penalty_{penalty}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'penalty' : 'MOC',
    'count' : 50,
    'budget_fracs' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], 
    'shots' : 1000, 
    'stepsize' : 2.00, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 100, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'CIA'
}

template = pd.read_csv('data/template_penalty.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], penalty = Config['penalty'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()


log_file, id = log._load_logfile()


Logfile data/penalty/Penalty_size_10_penalty_MOC_ansatz_CIA_stepsize_2.0_shots_1000.csv already exists


In [3]:

for i in tqdm.tqdm(range(Config['count']), desc = f"Generating matrices with size {Config['size']}"):
    matrix = np.random.uniform(size = (Config['size'], Config['size']), low = -1, high = 1) 
    matrix = (matrix.T + matrix) / 2
    for budget_frac in Config['budget_fracs']: 
    
        id += 1
        
        current_datetime = datetime.now()
        data = { 
            'Id (int)' : id, 
            'Valid (int)' : 1, 
            'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            'Matrix (mat)' : matrix,
            'Size (int)' : Config['size'],
            'Type (str)' : 'Penalty',
            'Rank (int)' : -1,
            'Density (flt)' : -1.0,
            'Stiffness (flt)' : -1.0,
            'Penalty (str)' : Config['penalty'],
            'Budget (flt)' : Config['size'] * budget_frac,
            'Test_solver (str)' : '-',
            'Test (flt)' : -1.0,
            'Test_state (vec)' : [],
            'Test_iterations (int)' : -1,
            'Test_time (flt)' : -1.0,
            'Train_solver (str)' : '-',
            'Preprocessing (str)' : '-',
            'Postprocessing (str)' : '-',
            'Train (flt)' : -1.0,
            'Train_state (vec)' : [],
            'Train_iterations (int)' : -1, 
            'Train_iterations_1 (int)' : -1,
            'Train_iterations_2 (int)' : -1,
            'Train_iterations_3 (int)' : -1,
            'Train_shots (int)': -1, 
            'Train_cost (flt)' : -1.0,
            'Train_time (flt)' : -1.0, 
            'Flags (str)' : '',
            'Baren (int)' : -1
        }
        
        log.log(data)

    
display(log.read(1))

Generating matrices with size 10:   8%|▊         | 4/50 [00:04<00:50,  1.10s/it]


KeyboardInterrupt: 

In [10]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])
prep = Preparator()

In [4]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        #print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''
            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h, constrain = True, k = data['Budget (flt)'])
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
        print(f'Matrix : {np.array(data['Matrix (mat)'])} | Budget : {data['Budget (flt)']} | Penalty : {Config['penalty']}')

        
        Q = data['Matrix (mat)']
        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

In [5]:

ids = range (1,351)
cores = 10
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    #display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



Matrix : [[-0.27048688  0.27974031 -0.3606227   0.17996606  0.55090435  0.17689486
  -0.2556318  -0.26324868  0.15759187  0.20913779]
 [ 0.27974031  0.24746174  0.22551017  0.51977914  0.6724946   0.40548559
   0.61680599 -0.30826908 -0.40562877 -0.46521858]
 [-0.3606227   0.22551017  0.5960818   0.44144673  0.17329611 -0.14006227
  -0.37312542 -0.17895852  0.30077278 -0.4467994 ]
 [ 0.17996606  0.51977914  0.44144673 -0.18692495 -0.01726326  0.32920755
   0.1915374   0.5100658  -0.00167804  0.59424396]
 [ 0.55090435  0.6724946   0.17329611 -0.01726326  0.44907367  0.30230389
  -0.13928718 -0.10718828 -0.3580383   0.11777793]
 [ 0.17689486  0.40548559 -0.14006227  0.32920755  0.30230389 -0.77965864
  -0.06161721 -0.0369873  -0.65433584 -0.35294728]
 [-0.2556318   0.61680599 -0.37312542  0.1915374  -0.13928718 -0.06161721
   0.77308157  0.27935397 -0.61174846 -0.21091338]
 [-0.26324868 -0.30826908 -0.17895852  0.5100658  -0.10718828 -0.0369873
   0.27935397 -0.76894082 -0.72685585  0.43

Matrix : [[-0.47366415  0.04277221  0.54612557  0.10511339 -0.25607298 -0.01765473
  -0.7375265  -0.34795996  0.22949273 -0.02418646]
 [ 0.04277221  0.99801783 -0.09789552 -0.66931984 -0.48907895 -0.12540131
   0.25123067  0.29711027 -0.56218133 -0.10052695]
 [ 0.54612557 -0.09789552  0.62393381  0.60803845  0.73350688 -0.10577924
   0.07214095 -0.59454643  0.13225501  0.28102222]
 [ 0.10511339 -0.66931984  0.60803845  0.72780977 -0.21665632 -0.15269858
  -0.33755705  0.36142642 -0.17166309 -0.49494826]
 [-0.25607298 -0.48907895  0.73350688 -0.21665632  0.71646917 -0.37591041
   0.75817479 -0.4917222   0.27589867  0.04730264]
 [-0.01765473 -0.12540131 -0.10577924 -0.15269858 -0.37591041 -0.29435803
  -0.10513328 -0.79356102  0.51307447  0.12929793]
 [-0.7375265   0.25123067  0.07214095 -0.33755705  0.75817479 -0.10513328
   0.60910249 -0.22842333  0.04270075 -0.13594888]
 [-0.34795996  0.29711027 -0.59454643  0.36142642 -0.4917222  -0.79356102
  -0.22842333 -0.77261958  0.27602398  0.0

Process ForkPoolWorker-2:
Process ForkPoolWorker-9:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:


KeyboardInterrupt: 

In [4]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        #Q = line['Matrix (mat)']
        
        trained_data = line.copy()
        Q = line['Matrix (mat)']
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False, constrain = True, k = line['Budget (flt)'], penalty = Config['penalty'])
        print(f'Trained id = {line['Id (int)']} with train {train} and budget {np.sum(np.where(train_bitstring > 0, 1, 0))} while test budget is {line['Budget (flt)']} | train_bitstring : {train_bitstring}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [5]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 5

solvers = [] 

depth = 1

ids = np.array(range(1, 351))
ids = ids.reshape(cores, len(ids) // cores)

template = pd.read_csv('data/template_penalty.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))



log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], penalty = Config['penalty'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()

data = [[log.read (id) for id in subrange] for subrange in ids]
#print(train_dataset)
# data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




Trained id = 71 with train -1.9874590222533943 and budget 6 while test budget is 2.0 | train_bitstring : [ 1 -1  1 -1  1 -1  1 -1  1  1]
Trained id = 1 with train -2.0546272331263373 and budget 3 while test budget is 2.0 | train_bitstring : [-1  1 -1 -1 -1 -1 -1  1  1 -1]
Trained id = 141 with train -2.217967625677317 and budget 3 while test budget is 2.0 | train_bitstring : [-1 -1 -1  1  1 -1  1 -1 -1 -1]
Trained id = 2 with train -2.553083635889257 and budget 4 while test budget is 3.0 | train_bitstring : [-1 -1 -1 -1 -1  1  1  1  1 -1]
Trained id = 142 with train -2.9818851623941582 and budget 4 while test budget is 3.0 | train_bitstring : [-1 -1 -1  1  1  1  1 -1 -1 -1]
Trained id = 211 with train -1.4831043329664946 and budget 3 while test budget is 2.0 | train_bitstring : [-1 -1  1 -1 -1 -1  1 -1 -1  1]
Trained id = 72 with train -1.941742525348843 and budget 3 while test budget is 3.0 | train_bitstring : [-1  1 -1 -1 -1  1  1 -1 -1 -1]
Trained id = 212 with train -2.036078667037

/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, list(line.keys())] = list(line.values())
/mnt/Computing/I_Fomin/QAOA/logger.py:232: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  log_file.loc[log_file['Id (int)'] == Id, 

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)


NameError: name 'processed_data' is not defined